# notes

## projections

- 4326 # wgs84 (for H3 compatibility)
    - H3 **does not use** wgs84/epsg4326, but it **does use** the "authalic radius" from that standard
    - lat/lng
- 3857 # web mercator (for plotting), but in meters!
- 2263 # new york (just what's used in the example below)

## geoviews

- tile options: https://github.com/holoviz/geoviews/blob/main/geoviews/tile_sources.py
    - also this, but i don't see a good way to programatically get all the options: https://geoviews.org/gallery/matplotlib/tile_sources.html 
- active tools: https://stackoverflow.com/questions/57767169/set-box-zoom-or-pan-or-wheel-zoom-as-default-in-using-holoviews-or-hvplot
- bokeh pallets: https://docs.bokeh.org/en/latest/docs/reference/palettes.html
- Bokeh "tap" tool seems to do something on this website, but i can't get the same functionality (isolating a single geometry) locally: https://geoviews.org/gallery/bokeh/new_york_boroughs.html

## folium

- seems to be most popular, but they make you manually determine the center and zoom level!
    - too much work. geoviews does it automatically
    - https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html

In [ ]:
import h3
import geopandas
import geodatasets
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df.crs

In [ ]:
df

In [ ]:
def foo(df, epsg):
    df = df.to_crs(epsg=epsg)
    
    ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k', column='BoroName', categorical=True, legend=True, legend_kwds={'loc': 'upper left'})
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    cx.add_basemap(ax, crs=df.crs, source=cx.providers.CartoDB.Positron)

In [ ]:
foo(df, 2263)

In [ ]:
foo(df, 4326)

In [ ]:
import geodatasets as gds
import geopandas as gpd
import geoviews as gv
import cartopy.crs as ccrs

gv.extension('bokeh');

In [ ]:
def bar(df, epsg):
    crs = ccrs.epsg(epsg)
    df = df.to_crs(crs)
    
    tiles = gv.tile_sources.CartoLight
    polys = gv.Polygons(df, vdims=['BoroName'], crs=df.crs)

    out = tiles * polys.opts(color='BoroName', cmap='Pastel1', tools=['hover'], width=1200, height=800, alpha=0.5, active_tools=['wheel_zoom'], xaxis=None, yaxis=None)

    return out

In [ ]:
bar(df, 3857)

# converting to cells

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df = df[['BoroName', 'geometry']]
df = df.to_crs(epsg=4326)
df

In [ ]:
df['boro_geo'] = df.BoroName

In [ ]:
res = 9

df1 = df.copy()
df2 = df.copy()

df1['boro_geo'] = df1.BoroName

df2.geometry = df2.geometry.apply(lambda x: h3.geo_to_cells(x, res=res)).apply(h3.cells_to_h3shape)
df2['boro_geo'] = df2.BoroName + ' H3'
df3 = pd.concat([df1, df2])
df3

In [ ]:
def bar(df, epsg):
    crs = ccrs.epsg(epsg)
    df = df.to_crs(crs)
    
    tiles = gv.tile_sources.CartoLight
    polys = gv.Polygons(df, vdims=['BoroName', 'boro_geo'], crs=df.crs)

    out = tiles * polys.opts(color='BoroName', cmap='Pastel1', tools=['hover'], width=1200, height=800, alpha=0.5, active_tools=['wheel_zoom'], xaxis=None, yaxis=None)

    return out

In [ ]:
bar(df3, 3857)